In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv
/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv


****Below is first model in which like missing values are replaced by overall median****

In [87]:
import pandas as pd 
df=pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv")
df.fillna(df.median(numeric_only=True),inplace=True)
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Drop ID column
df.drop(columns=['ID'], inplace=True)

# Encode class column (if it's categorical)
label_encoder = LabelEncoder()
df['class'] = label_encoder.fit_transform(df['class'])

# Split into features and target
X = df.drop(columns=['class'])
y = df['class']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Fit multinomial logistic regression
model = LogisticRegression(
    multi_class='multinomial',
    solver='lbfgs',
    max_iter=1000
)
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Classification report with all original class labels
print(classification_report(
    y_test,
    y_pred,
    labels=list(range(len(label_encoder.classes_))),
    target_names=label_encoder.classes_
))
model.score(X_train,y_train)
model.score(X_test,y_test)
X_test
test_set=pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")
ids=test_set["ID"]
test_set.drop(columns=["ID"],inplace=True)
x1=test_set
y1=model.predict(x1)
y1
pred_class=label_encoder.inverse_transform(y1)
final_dataset=pd.DataFrame({"ID":ids,"class":pred_class})
final_dataset["class"].value_counts()
final_dataset.to_csv("Prediction.csv",index=False)
print(label_encoder.classes_)
df
model.score(X_train,y_train)
#model.score(X_test,y_test)

              precision    recall  f1-score   support

        farm       0.77      0.80      0.79       168
      forest       0.98      0.99      0.98      1232
       grass       0.62      0.46      0.53        39
  impervious       0.75      0.82      0.79       134
     orchard       0.50      0.17      0.25         6
       water       0.92      0.52      0.67        21

    accuracy                           0.93      1600
   macro avg       0.76      0.63      0.67      1600
weighted avg       0.93      0.93      0.93      1600

['farm' 'forest' 'grass' 'impervious' 'orchard' 'water']


0.94796875

****So like for second model what we tried to is like replace the missing values with median of respective values for each class and thats what worked better***

In [62]:
dataset=pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv")
dataset1=dataset[pd.notnull(dataset["20150720_N"])]
class_median=pd.DataFrame(dataset1.groupby("class")[column].median() for column in dataset.columns.values[3:])
class_median.reset_index()
def datafix(row):
    columns=dataset.columns.values[3:]
    for column in columns:
      if pd.isna(row[column]) and row["class"]=="farm":
          row[column]=class_median.loc[column,"farm"]
      if pd.isna(row[column]) and row["class"]=="forest":
          row[column]=class_median.loc[column,"forest"]
      if pd.isna(row[column]) and row["class"]=="grass":
          row[column]=class_median.loc[column,"grass"]
      if pd.isna(row[column]) and row["class"]=="impervious":
          row[column]=class_median.loc[column,"impervious"]
      if pd.isna(row[column]) and row["class"]=="water":
          row[column]=class_median.loc[column,"water"]
      if pd.isna(row[column]) and row["class"]=="orchard":
          row[column]=class_median.loc[column,"orchard"]
    return row
dataset=dataset.apply(datafix,axis="columns")

***Standardization was also done along with better replacement of missing values***

In [89]:
data=dataset.copy()
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

# Drop ID column

# Encode class column (if it's categorical)
label_encoder3 = LabelEncoder()
data['class'] = label_encoder3.fit_transform(data['class'])

# Split into features and target
X = data.iloc[:,3:]
y = data['class']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Fit multinomial logistic regression
model = LogisticRegression(
    multi_class='multinomial',
    solver='lbfgs',
    max_iter=2000
)
model.fit(X_train_scaled, y_train)
# Predict on test set
y_pred = model.predict(X_test_scaled)

# Classification report with all original class labels
'''print(classification_report(
    y_test,
    y_pred,
    labels=list(range(len(label_encoder3.classes_))),
    target_names=label_encoder3.classes_
))'''
model.score(X_train_scaled,y_train)
model.score(X_test_scaled,y_test)
import matplotlib.pyplot as plt
import seaborn as sns
y_test_decoded=label_encoder3.inverse_transform(y_test)
y_pred_decoded=label_encoder3.inverse_transform(y_pred)
cm=confusion_matrix(y_test_decoded,y_pred_decoded)
plt.figure(figsize=(10, 8))
'''sns.heatmap(cm, 
            annot=True,  # Show numbers in cells
            fmt='d',     # Format as integers
            cmap='Blues',
            xticklabels=label_encoder3.classes_,
            yticklabels=label_encoder3.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()'''
old_pred=pd.read_csv("/kaggle/input/prediction1/Prediction.csv")
old_pred["class"].value_counts()
new_dataset=pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")
ids=new_dataset["ID"]
x_new=new_dataset.iloc[:,2:]
x_new
x_scaled=scaler.transform(x_new)
x_scaled
y_new=model.predict(x_scaled)
y_pred_new=label_encoder3.inverse_transform(y_new)
y_pred_new
dfinal=pd.DataFrame({"ID":ids,"class":y_pred_new})
old_pred["class"].value_counts()
dfinal["class"].value_counts()
dfinal.to_csv("NewPrediction.csv",index=False)

<Figure size 1000x800 with 0 Axes>

****Below is some like analysis of previous versus new predictions****

In [90]:
# Load both prediction files
old_predictions = pd.read_csv("Prediction.csv")  # Previous prediction with score of 0.53136
new_predictions = pd.read_csv("NewPrediction.csv")      # New prediction

# Checking the different predictions
predictions_changed = (old_predictions['class'] != new_predictions['class']).sum()
total_predictions = len(old_predictions)

print(f"Total predictions: {total_predictions}")
print(f"Changed predictions: {predictions_changed}")
print(f"Percentage changed: {predictions_changed/total_predictions*100:.2f}%")

# Loooking where they are different
differences = old_predictions[old_predictions['class'] != new_predictions['class']]
print(f"\nFirst 10 differences:")
print(differences[['ID']].head(10))

# Also checking difference and distribution according to count
print("\nOld model class distribution:")
print(old_predictions['class'].value_counts().sort_index())
print("\nNew model class distribution:")
print(new_predictions['class'].value_counts().sort_index())

Total predictions: 2845
Changed predictions: 1017
Percentage changed: 35.75%

First 10 differences:
    ID
0    1
2    3
3    4
4    5
5    6
8    9
9   10
15  16
16  17
18  19

Old model class distribution:
class
farm             8
forest        2588
grass            1
impervious      27
orchard          2
water          219
Name: count, dtype: int64

New model class distribution:
class
farm           459
forest        1736
grass          123
impervious     418
orchard          7
water          102
Name: count, dtype: int64


In [91]:
# Creating a comparison dataframe
comparison = pd.DataFrame({
    'ID': old_predictions['ID'],
    'old_pred': old_predictions['class'],
    'new_pred': new_predictions['class'],
    'changed': old_predictions['class'] != new_predictions['class']
})

# Looking at sample changes
print("Sample prediction changes:")
changed_samples = comparison[comparison['changed']].head(10)
for _, row in changed_samples.iterrows():
    print(f"ID {row['ID']}: {row['old_pred']} → {row['new_pred']}")

# Checking that new model is confident in which of the classes
print("\nClass changes summary:")
change_summary = comparison[comparison['changed']].groupby(['old_pred', 'new_pred']).size()
print(change_summary)

Sample prediction changes:
ID 1: forest → farm
ID 3: forest → orchard
ID 4: forest → farm
ID 5: forest → orchard
ID 6: forest → farm
ID 9: forest → farm
ID 10: forest → farm
ID 16: forest → farm
ID 17: forest → orchard
ID 19: forest → orchard

Class changes summary:
old_pred    new_pred  
farm        impervious      8
forest      farm          425
            grass         123
            impervious    307
            orchard         7
grass       impervious      1
impervious  water          13
orchard     impervious      1
            water           1
water       farm           34
            forest         10
            impervious     87
dtype: int64


In [93]:
print("=== MODEL COMPARISON ===")
print(f"Old submission score: 0.53136")
print(f"New model validation accuracy: {model.score(x_scaled, y_new):.5f}")
print(f"Predictions changed: {predictions_changed}/{total_predictions}")

if model.score(X_test_scaled, y_test) > 0.53136:
    print("✅ Validation accuracy improved - good sign!")
else:
    print("⚠️  Validation accuracy not clearly better")

if predictions_changed > total_predictions * 0.1:  # More than 10% changed
    print("✅ Significant prediction changes - could be much better!")
elif predictions_changed > 0:
    print("✅ Some changes - might be slightly better")
else:
    print("❌ No changes - same score expected")

=== MODEL COMPARISON ===
Old submission score: 0.53136
New model validation accuracy: 1.00000
Predictions changed: 1017/2845
✅ Validation accuracy improved - good sign!
✅ Significant prediction changes - could be much better!
